In [1]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *

# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Loading your IBM Quantum account(s)
service = QiskitRuntimeService(channel="ibm_quantum")

# Invoke a primitive. For more details see https://qiskit.org/documentation/partners/qiskit_ibm_runtime/tutorials.html
# result = Sampler("ibmq_qasm_simulator").run(circuits).result()

In [ ]:
from qiskit_nature.second_q.drivers import PySCFDriver
from pyscf import scf, gto
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitConverter


backend = service.get_backend('ibmq_belem')

def get_qubit_op(d):

    h2 = MoleculeInfo(["H", "H"], 
                        [(0.0, 0.0, 0.0), (0.0, 0.0, d)], 
                        charge= 0,
                        multiplicity= 1 # 2*spin + 1,
                        )

    driver = PySCFDriver.from_molecule(h2 , basis="sto3g")
    
    problem = driver.run() 
    
    num_particles=problem.num_particles
    
    num_orbitals=int(problem.num_spatial_orbitals)
    
    hamiltonian = problem.hamiltonian
    
    second_q_op = hamiltonian.second_q_op()
    
    mapper=JordanWignerMapper()
    
    converter=QubitConverter(mapper)
    
    qubit_op=converter.convert(second_q_op)

    return qubit_op,problem,converter, num_particles, num_orbitals

def hf_ener(d):
    
    mol = gto.M(
      verbose=0,
      atom=[['H',(0, 0, 0)], 
            ['H',(0, 0, d)]],
      basis="sto3g",
      charge = 0
    )
    
    mf = scf.RHF(mol).x2c().run()
    hf = mf.kernel()

    return hf

from qiskit.algorithms.optimizers import SPSA
from qiskit_nature.second_q.algorithms import GroundStateEigensolver, NumPyMinimumEigensolverFactory, NumPyEigensolverFactory, ExcitedStatesEigensolver, QEOM
from qiskit_aer.primitives import Estimator as AerEstimator
from qiskit_nature.second_q.circuit.library import HartreeFock
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.circuit.library import EfficientSU2

import numpy as np

def filter_criterion(eigenstate, eigenvalue, aux_values): # This filter function must be added to get S=1 states.
    
    return np.isclose(aux_values["ParticleNumber"][0], 2.0)


def np_sol(problem, converter):
    
    solver_ground = NumPyMinimumEigensolverFactory()
    
    calc_gs = GroundStateEigensolver(converter, solver_ground)
    
    np_result_gs = calc_gs.solve(problem)
    
    np_result_gs = np_result_gs.total_energies[0].real
    
    return np_result_gs


spsa = SPSA(maxiter=5)

def vqe_sol(num_orbitals,num_particles,converter,problem):

    
    init_state = HartreeFock(num_orbitals,num_particles, converter)
    
    two_local = EfficientSU2(num_qubits=4, entanglement='linear', initial_state=init_state)
    
    vqe_spsa = VQE(estimator, two_local, spsa)
    
    vqe_solution_two_local = vqe_spsa.compute_minimum_eigenvalue(qubit_op)
    
    vqe_result_two_local = problem.interpret(vqe_solution_two_local).total_energies[0].real
    
    hf_result = hf_ener(d)            
    
    return hf_result, vqe_result_two_local


distances = np.loadtxt('distances_shortened_part2.txt') # Loads data (in a.u.)
distances = distances * 0.529177249 # In Angstrom
ref_ener_gs = np.loadtxt('exact_ener.txt')

vqe_energies = []
hf_energies = []
np_energies = []

with Session(backend=backend):
    
    estimator = Estimator(options={'shots':int(1e1)})

    for d in distances:
    
        (qubit_op,problem,converter, num_particles, num_orbitals) = get_qubit_op(d)
        
        (hf_result, vqe_result_two_local) = vqe_sol(num_orbitals,num_particles,converter,problem)
    
        (np_result_gs) = np_sol(problem, converter)
    
    
        vqe_energies.append(vqe_result_two_local)  
    
        hf_energies.append(hf_result)
    
        np_energies.append(np_result_gs) 
    
    
        print(f"Interatomic Distance: {np.round(d, 2)}")


    print("All energies have been calculated")


h2_real_qc = {}
h2_real_qc['distances']= distances
h2_real_qc['ref_ener_gs']= ref_ener_gs
h2_real_qc['vqe_energies'] = vqe_energies
h2_real_qc['hf_energies'] = hf_energies
h2_real_qc['np_energies'] = np_energies 

np.savez("h2_real_qc_part2", **h2_real_qc)
